# Prediction of Sharpe ratio for blends of quantitative srategies

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
data = pd.read_csv("Data/Training_Input_2dx8C9Q.csv")
target = pd.read_csv("Data/Training_Output_IJhBXtA.csv")

In [13]:
test = pd.read_csv("Data/Testing_Input_dPKY3Rf.csv")

In [6]:
data.shape

(10000, 218)

In [7]:
target.shape

(10000, 2)

In [8]:
test.shape

(4450, 218)

In [15]:
data = data.set_index("ID")
target = target.set_index("ID")

In [18]:
data.head()

,weight_I_1,weight_I_2,weight_I_3,weight_I_4,weight_I_5,weight_I_6,weight_I_7,I_1_lag_20,I_1_lag_19,I_1_lag_18,...,X_3_lag_9,X_3_lag_8,X_3_lag_7,X_3_lag_6,X_3_lag_5,X_3_lag_4,X_3_lag_3,X_3_lag_2,X_3_lag_1,X_3_lag_0
ID,,,,,,,,,,,,,,,,,,,,,
0,0.15,0.00,0.05,0.80,0.00,0.0,0.00,100.0,100.047398,100.058480,...,101.383783,102.054669,102.375596,103.148605,103.148605,103.046483,103.075701,103.134043,103.221509,103.338192
1,0.00,0.00,0.00,0.40,0.25,0.0,0.35,100.0,99.912339,99.960558,...,100.911142,100.938707,100.993926,101.132016,100.745489,100.524617,100.303743,100.276090,100.303743,100.554527
2,0.85,0.00,0.00,0.15,0.00,0.0,0.00,100.0,99.481681,100.199593,...,100.373084,100.581716,100.313489,100.790251,101.013756,100.686030,100.686030,100.060233,99.747384,99.970889
3,0.00,0.00,0.70,0.05,0.25,0.0,0.00,100.0,100.124618,100.023384,...,100.844136,101.040072,101.055122,101.567682,101.703322,101.974603,101.733422,101.838963,102.080144,101.688272
4,0.00,0.55,0.05,0.00,0.00,0.0,0.40,100.0,100.000000,99.407717,...,99.665093,99.482389,99.604192,100.030499,99.847797,100.426310,100.426310,100.822217,100.913521,100.852619


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 217 entries, weight_I_1 to X_3_lag_0
dtypes: float64(217)
memory usage: 16.6 MB


In [17]:
target.head()

,Target
ID,
0,-12.007941
1,2.294867
2,0.652308
3,2.412364
4,8.517471


In [21]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Target  10000 non-null  float64
dtypes: float64(1)
memory usage: 156.2 KB


In [19]:
test.head()

,ID,weight_I_1,weight_I_2,weight_I_3,weight_I_4,weight_I_5,weight_I_6,weight_I_7,I_1_lag_20,I_1_lag_19,...,X_3_lag_9,X_3_lag_8,X_3_lag_7,X_3_lag_6,X_3_lag_5,X_3_lag_4,X_3_lag_3,X_3_lag_2,X_3_lag_1,X_3_lag_0
0,10000,0.0,0.4,0.0,0.60,0.00,0.00,0.0,100.0,100.035108,...,99.476884,99.385263,99.503013,98.927246,98.704811,98.547784,98.573913,98.665534,98.613191,99.149680
1,10001,0.3,0.0,0.2,0.50,0.00,0.00,0.0,100.0,99.396127,...,101.381133,101.480672,101.194476,101.306518,100.871012,100.522624,101.045166,101.157209,101.169631,100.846088
2,10002,0.0,0.2,0.0,0.80,0.00,0.00,0.0,100.0,101.592811,...,99.206743,99.020803,99.008428,99.095210,99.529043,99.516588,99.590996,100.285096,100.173565,100.099157
3,10003,0.0,0.0,0.0,0.05,0.85,0.00,0.1,100.0,99.732372,...,99.865944,98.839654,98.211795,97.571802,97.330318,97.137131,97.487341,97.861584,97.801232,97.777122
4,10004,0.0,0.0,0.0,0.10,0.45,0.45,0.0,100.0,99.962739,...,98.067865,98.188662,98.140360,98.454292,98.949460,98.442236,98.176606,98.357686,98.707943,98.369820


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4450 entries, 0 to 4449
Columns: 218 entries, ID to X_3_lag_0
dtypes: float64(217), int64(1)
memory usage: 7.4 MB


In [24]:
# Missing values ?
print(True in data.isna(), True in target.isna(), True in test.isna())

False False False


In [24]:
# Let's see the columns to understand the construction of the dataset. 
# It begins with lag_20 and finishes with lag_0 for the 10 time series. 
# So lag_20 represent the fist day of a month set to a value of "100"
for i in data.columns:
    print(i)

ID
weight_I_1
weight_I_2
weight_I_3
weight_I_4
weight_I_5
weight_I_6
weight_I_7
I_1_lag_20
I_1_lag_19
I_1_lag_18
I_1_lag_17
I_1_lag_16
I_1_lag_15
I_1_lag_14
I_1_lag_13
I_1_lag_12
I_1_lag_11
I_1_lag_10
I_1_lag_9
I_1_lag_8
I_1_lag_7
I_1_lag_6
I_1_lag_5
I_1_lag_4
I_1_lag_3
I_1_lag_2
I_1_lag_1
I_1_lag_0
I_2_lag_20
I_2_lag_19
I_2_lag_18
I_2_lag_17
I_2_lag_16
I_2_lag_15
I_2_lag_14
I_2_lag_13
I_2_lag_12
I_2_lag_11
I_2_lag_10
I_2_lag_9
I_2_lag_8
I_2_lag_7
I_2_lag_6
I_2_lag_5
I_2_lag_4
I_2_lag_3
I_2_lag_2
I_2_lag_1
I_2_lag_0
I_3_lag_20
I_3_lag_19
I_3_lag_18
I_3_lag_17
I_3_lag_16
I_3_lag_15
I_3_lag_14
I_3_lag_13
I_3_lag_12
I_3_lag_11
I_3_lag_10
I_3_lag_9
I_3_lag_8
I_3_lag_7
I_3_lag_6
I_3_lag_5
I_3_lag_4
I_3_lag_3
I_3_lag_2
I_3_lag_1
I_3_lag_0
I_4_lag_20
I_4_lag_19
I_4_lag_18
I_4_lag_17
I_4_lag_16
I_4_lag_15
I_4_lag_14
I_4_lag_13
I_4_lag_12
I_4_lag_11
I_4_lag_10
I_4_lag_9
I_4_lag_8
I_4_lag_7
I_4_lag_6
I_4_lag_5
I_4_lag_4
I_4_lag_3
I_4_lag_2
I_4_lag_1
I_4_lag_0
I_5_lag_20
I_5_lag_19
I_5_lag_18
I_5

In [26]:
# There is up to 50 different samples (different weights) for the same time serie of 26 days 
# (21 days of training and 5 days of prediction)
# given a train test of 10 000 sample, if there is exactly 50 different, we should obtain :
# 10 000/50 = 200 different time series
# Let's try to verify the exact number of different samples for a given time serie :

In [77]:
# For macro indicator X_1 we can find our same 200 time series starting with lag_18 
for i in range(0,21):
    print(i,data["X_1_lag_"+str(i)].nunique())

0 200
1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 200
14 200
15 200
16 200
17 200
18 200
19 193
20 1


In [70]:
for i in range(0,21):
    print(i,data["I_7_lag_"+str(i)].nunique())

0 200
1 200
2 200
3 200
4 196
5 194
6 192
7 191
8 191
9 184
10 181
11 181
12 179
13 174
14 173
15 172
16 171
17 169
18 164
19 157
20 1
